<a href="https://colab.research.google.com/github/ashok-arora/ML-Zoomcamp/blob/main/homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Zoomcamp Homework 3](#https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/03-classification/homework.md)

In [56]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv

--2021-09-26 09:03:47--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘AB_NYC_2019.csv.2’

AB_NYC_2019.csv.2   100%[===================>]   6.75M  --.-KB/s    in 0.1s    

2021-09-26 09:03:47 (65.6 MB/s) - ‘AB_NYC_2019.csv.2’ saved [7077973/7077973]



In [57]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import mutual_info_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [58]:
df = pd.read_csv('./AB_NYC_2019.csv')
print(df.columns)
df.head()

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [59]:
df = df[['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']]

In [60]:
df = df.fillna(0)

## Answer 1: Mode

In [61]:
df.neighbourhood_group.mode()

0    Manhattan
dtype: object

## Answer 2: Correlation Matrix


In [62]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)
print(len(df), len(df_train), len(df_val), len(df_test))

48895 29337 9779 9779


In [63]:
y_train = df_train.price.values
y_val = df_val.price.values

In [64]:
del df_train['price']
del df_val['price']

In [65]:
df_train.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [66]:
c = df_train.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)

rows_to_skip = len(df_train.corr())
print(so[rows_to_skip:rows_to_skip+3])

number_of_reviews               reviews_per_month    0.590374
reviews_per_month               number_of_reviews    0.590374
calculated_host_listings_count  availability_365     0.225913
dtype: float64


## Answer 3: Mutual Information Score


In [67]:
above_average = (y_train >= 152).astype(int)
above_average

array([0, 0, 0, ..., 1, 0, 0])

In [68]:
from sklearn.metrics import mutual_info_score
round(mutual_info_score(above_average, df_train.neighbourhood_group), 2)

0.05

In [69]:
round(mutual_info_score(above_average, df_train.room_type), 2)

0.14

## Answer 4: Logistic Regression


In [70]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
df_train_transformed = dv.fit_transform(train_dict)

In [71]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42, max_iter=2000)
model.fit(df_train_transformed, above_average)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
val_dict = df_val.to_dict(orient='records')
df_val_transformed = dv.transform(val_dict)

y_val_binary = (y_val >= 152).astype(int)

In [73]:
round(model.score(df_val_transformed, y_val_binary), 2)

0.79

## Answer 5: Feature Elimination

In [74]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
df_train_transformed = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
df_val_transformed_full = dv.transform(val_dict)
df_val_binary = (y_val >= 152).astype(int)

full_model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42, max_iter=2000)
full_model.fit(df_train_transformed, above_average)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
features = ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


In [76]:
diffs = {}
for feature in features:
    dv = DictVectorizer(sparse=False)
    train_dict = df_train.drop([feature], axis=1).to_dict(orient='records')
    df_train_transformed = dv.fit_transform(train_dict)
    
    val_dict = df_val.drop([feature], axis=1).to_dict(orient='records')
    df_val_transformed = dv.transform(val_dict)

    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42, max_iter=2000)
    model.fit(df_train_transformed, above_average)

    diffs[feature] = full_model.score(df_val_transformed_full, y_val_binary) - model.score(df_val_transformed, y_val_binary)

In [77]:
min(diffs.items(), key=lambda x: x[1])

('number_of_reviews', -0.001329379282135168)

## Answer 6: Ridge Regression

In [78]:
y_train = np.log1p(y_train)
y_train

array([4.60517019, 4.06044301, 4.26267988, ..., 5.70378247, 4.18965474,
       4.53259949])

In [79]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
df_train_transformed = dv.fit_transform(train_dict)
val_dict = df_val.to_dict(orient='records')
df_val_transformed = dv.transform(val_dict)

scores = {}
for alpha in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha)

    model.fit(df_train_transformed, y_train)

    scores[alpha] = (round(mean_squared_error(model.predict(df_val_transformed), y_val, squared=False), 3))

In [80]:
min(scores.items(), key=lambda x: x[1])

(0.01, 268.914)